In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
import scipy.io.wavfile as wavf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
from tqdm import tqdm
import os

In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import pickle
from pydub import AudioSegment

C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
def getSplitIndexes(filename):
    filename='test/fullSentence/'+filename
    
    def isNaN(x):
        return x != x
    signal,sample_rate = librosa.load(filename)
    
    for i in range(len(signal)):
        if signal[i] < 0:
            signal[i] = -signal[i]
    
    index_list = []
    j = 0
    while j < len(signal):
        index_list.append(j)
        j = j + 1
        
    df = pd.DataFrame(list(zip(index_list, signal)),columns =['time', 'signal'])
    
    df['mov_avg'] = df['signal'].rolling(10000).sum()
    
    smooth_list = df['mov_avg'].tolist()
    #print(len(smooth_list))
    
    minimas = []
    i = 1
    while isNaN(smooth_list[i]):
        i = i + 1
    
    #print(i)
    window = 15000

    while i + window < len(smooth_list):
        #print("hi")
        m = min(smooth_list[i:i+window])
        idx = smooth_list.index(m)
        #print(m,idx,smooth_list[i],smooth_list[i+window-1])
        
        if m != smooth_list[i] and m != smooth_list[i+window-1]:
            if len(minimas) != 0  and abs(idx - minimas[-1][0]) > 5000:  
                minimas.append([idx,m])
            if len(minimas) == 0:
                minimas.append([idx,m])
    
        i = i + window

    print(minimas)
    return minimas
    
        
    
    
    
    
    

In [5]:
def splitWaveFile(fileName):
    splitIndexes = getSplitIndexes(fileName)
    signal,sample_rate = librosa.load('test/fullSentence/'+fileName)
    splitIndexes.insert(0, [0,0])
    for i in range(1,len(splitIndexes)):
        subSignal=signal[splitIndexes[i-1][0]:splitIndexes[i][0]]
        subSignal = np.array(subSignal* (1<<15), dtype=np.int16)
        audio_segment = AudioSegment(
            subSignal.tobytes(), 
            frame_rate=sample_rate,
            sample_width=subSignal.dtype.itemsize, 
            channels=1
        )
        audio_segment.export("test/original/"+fileName.split(".")[0]+"_"+str(i)+".wav",format="wav")
    
    subSignal=signal[splitIndexes[-1][0]:]
    subSignal = np.array(subSignal* (1<<15), dtype=np.int16)
    audio_segment = AudioSegment(
            subSignal.tobytes(), 
            frame_rate=sample_rate,
            sample_width=subSignal.dtype.itemsize, 
            channels=1
    )
    audio_segment.export("test/original/"+fileName.split(".")[0]+"_"+str(len(splitIndexes))+".wav",format="wav")
    return len(splitIndexes)

In [6]:
def preProcess(fileName):
    def originalToScaled(fileName):
        signal,sample_rate=librosa.load("test/original/"+fileName)
        obsMax = max(signal)
        obsMin = 0

        newMax = 0.1
        newMin = 0

        new_signal = []

        for ele in signal:
            ele = abs(ele)
            x = (newMax - newMin)/(obsMax - obsMin)*(ele - obsMin) + newMin
            new_signal.append(x)

        new_signal = np.array(new_signal)

        for i in range(len(signal)):
            if signal[i] < 0:
                new_signal[i] = 0 - new_signal[i]

        new_signal = np.array(new_signal* (1<<15), dtype=np.int16)

        audio_segment = AudioSegment(
        new_signal.tobytes(), 
        frame_rate=sample_rate,
        sample_width=new_signal.dtype.itemsize, 
        channels=1
        )

        audio_segment.export("test/scaled/"+fileName,format="wav")
        
    
    def scaledToTrimmed(fileName):
        def detect_leading_silence(sound, silence_threshold=-45.0, chunk_size=10):
            '''
            sound is a pydub.AudioSegment
            silence_threshold in dB
            chunk_size in ms

            iterate over chunks until you find the first one with sound
            '''
            trim_ms = 0 # ms

            assert chunk_size > 0 # to avoid infinite loop
            while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
                trim_ms += chunk_size

            return trim_ms

        sound = AudioSegment.from_file("test/scaled/"+fileName, format="wav")

        start_trim = detect_leading_silence(sound)
        end_trim = detect_leading_silence(sound.reverse())

        duration = len(sound)    
        trimmed_sound = sound[start_trim:duration-end_trim]
        trimmed_sound.export("test/trimmed/"+fileName, format="wav")

    def trimmedToSpeedChange(fileName):
        targetLen=20000
        sound = AudioSegment.from_file("test/trimmed/"+fileName)
        sample = sound.get_array_of_samples()
        sample = np.array(sample)
        sampleLen=len(sample)

        def speed_change(sound, speed):

            sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
                "frame_rate": int(sound.frame_rate * speed)
            })
            sound_with_altered_frame_rate.export('test/speedChange/'+fileName, format ="wav")

        slow_sound = speed_change(sound, sampleLen/targetLen)
    
    originalToScaled(fileName)
    scaledToTrimmed(fileName)
    trimmedToSpeedChange(fileName)

In [7]:
def wordPredictor(sentence):
    sol = []
    if len(sentence.split()) == 1:
        encoded_text = tokenizerOne.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=1, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelOne.predict(pad_encoded)[0]).argsort()[-5:][::-1]:
          pred_word = tokenizerOne.index_word[i]
          sol.append(pred_word)
            
    elif len(sentence.split()) == 2:
        encoded_text = tokenizerTwo.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=2, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelTwo.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
          pred_word = tokenizerTwo.index_word[i]
          sol.append(pred_word)
            
    else:
        encoded_text = tokenizerThree.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=3, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelThree.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
          pred_word = tokenizerThree.index_word[i]
          sol.append(pred_word)
            
    return sol
        

In [8]:
def predict(chunk_data,classes):
    df = pd.read_csv(r"references.csv")
    print("classes = ", classes)

    def features_extractor(file):
        audio,sample_rate = librosa.load(file , res_type = 'kaiser_fast')
        mfccs_features = librosa.feature.mfcc(y = audio,sr = sample_rate, n_mfcc=40)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis = 0)
        return mfccs_scaled_features
    
    extracted_features = []
    #classes = ["plumbing","iro","service","when"]
    #audio_path_dir =str("C:Users\sai\Desktop\Capstone project\DL\coding\audio_data")
    cwd = os.getcwd()
    for i in df.index:
        class_name = df['class_name'][i]

        audio_file_name = str(df['file_name'][i])

        if class_name in classes:
            #audio_path = cwd + "\\audio_data\\" + audio_file_name + ".wav"
            audio_path = "train/speedChange/" +   audio_file_name
            #print(audio_path)
            data = features_extractor(audio_path)
            extracted_features.append([class_name,data])
            
    df_extracted_features = pd.DataFrame(extracted_features,columns = ['class_name' , 'feature_vector'])
    
    mini_df = df_extracted_features
    
    X = np.array(mini_df['feature_vector'].tolist())
    y = np.array(mini_df['class_name'].tolist())
    
    #y = np.array(pd.get_dummies(y))
    labelencoder=LabelEncoder()
#     print('Y before',y)
    y=labelencoder.fit_transform(y)
#     print('Y integer',y)
    onehot_encoder = OneHotEncoder(sparse=False)
    y = y.reshape(len(y), 1)
    y = onehot_encoder.fit_transform(y)
#     y=to_categorical(labelencoder.fit_transform(y))
#     labelencoder.fit(y)
#     y=labelencoder.transform(y)
#     y=np.array([[i]for i in y])
#     print('Y after',y)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)
#     print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    
    
    
    num_labels=y.shape[1]
    model=Sequential()
    ###first layer
    model.add(Dense(100,input_shape=(40,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    ###second layer
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    ###third layer
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    ###final layer
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
    from tensorflow.keras.callbacks import ModelCheckpoint
    from datetime import datetime 
    num_epochs = 500
    num_batch_size = 4

    model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=0)
    test_accuracy=model.evaluate(X_test,y_test,verbose=0)
    print("Test Accuracy",test_accuracy[1])
    #Preditcting the class
    
    mfccs_features = librosa.feature.mfcc(y = chunk_data , sr=22050, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    
    predicted_label = model.predict_classes(mfccs_scaled_features)
    predict_probability=model.predict(mfccs_scaled_features)
    print("Predcited Label",predicted_label)
    print("Predcited Prob",predict_probability)

    prediction_class = labelencoder.inverse_transform(predicted_label) 
    print("Predcited Class",prediction_class[0])
    return prediction_class[0]    
    
    

In [9]:
def speech_to_text(filename,totalFiles):
    cur_sentence = "s"
    for i in range(1,totalFiles+1):
        X,sample_rate=librosa.load('test/speedChange/'+fileName.split(".")[0]+"_"+str(i)+".wav" , res_type = 'kaiser_fast')
        classes = wordPredictor(cur_sentence)
        cur_sentence = cur_sentence + " " + predict(X,classes)
    return cur_sentence
    
    
    

In [10]:
modelOne = load_model("../wordPredictor/modelOne/modelOne.h5")
modelTwo = load_model("../wordPredictor/modelTwo/modelTwo.h5")
modelThree = load_model("../wordPredictor/modelThree/modelThree.h5")

In [11]:
with open('../wordPredictor/modelOne/tokenizerOne.tokenizer', 'rb') as config_dictionary_file:
    tokenizerOne = pickle.load(config_dictionary_file)

with open('../wordPredictor/modelTwo/tokenizerTwo.tokenizer', 'rb') as config_dictionary_file:
    tokenizerTwo = pickle.load(config_dictionary_file)

with open('../wordPredictor/modelThree/tokenizerThree.tokenizer', 'rb') as config_dictionary_file:
    tokenizerThree = pickle.load(config_dictionary_file)

    

In [12]:
fileName = 'ad_58_1.wav'   
totalFiles=splitWaveFile(fileName)

    


[[37809, 13.05748274519064], [65385, 12.928407609132792], [89288, 14.55607221989402]]


In [13]:
for i in range(1,totalFiles+1):
    preProcess(fileName.split(".")[0]+"_"+str(i)+".wav")

In [14]:
final_output = speech_to_text(fileName,totalFiles)
print(final_output)

[1] [[1]]
classes =  ['what', 'hathira', 'hathiradha', 'are', 'nanna']
Test Accuracy 0.8666666746139526
Predcited Label [3]
Predcited Prob [[2.7357256e-16 1.4512064e-10 7.0856374e-13 1.0000000e+00 2.2851456e-18]]
Predcited Class nanna
[1, 21] [[ 1 21]]


C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


classes =  ['hathira', 'next', 'traffic']
Test Accuracy 0.8888888955116272
Predcited Label [2]
Predcited Prob [[9.1540682e-07 1.1221816e-09 9.9999905e-01]]
Predcited Class traffic
[1, 30, 56] [[ 1 30 56]]


C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


classes =  ['stations', 'plumbing', 'kolaayi']
Test Accuracy 0.8888888955116272
Predcited Label [1]
Predcited Prob [[4.7260551e-07 9.9999952e-01 5.4003734e-14]]
Predcited Class plumbing
[1, 30, 56, 35] [[30 56 35]]
classes =  ['yavdu', 'service', 'open']


C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Test Accuracy 0.29629629850387573
Predcited Label [2]
Predcited Prob [[0.3330005  0.31862897 0.34837055]]
Predcited Class yavdu
s nanna traffic plumbing yavdu


C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
plt.figure(figsize=(14,5))
signal,sample_rate=librosa.load("train/original/dr_nanna_2.wav")
lenSignal=len(signal)
librosa.display.waveplot(signal,sr=sample_rate)
lenSignal

In [ ]:
plt.figure(figsize=(14,5))
signal,sample_rate=librosa.load("train/trimmed/dr_nanna_2.wav")
lenSignal=len(signal)
librosa.display.waveplot(signal,sr=sample_rate)
lenSignal

In [ ]:
plt.figure(figsize=(14,5))
signal,sample_rate=librosa.load("test/speedChange/testaudio1.wav")
lenSignal=len(signal)
librosa.display.waveplot(signal,sr=sample_rate)


In [ ]:
preProcess("dr_nanna_1.wav")